# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install metasynth`

In [1]:
# %pip install metasynth

In [2]:
# import required packages
import datetime as dt
import pandas as pd
from metasynth import MetaDataset
from utils import get_demonstration_fp

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [3]:
demonstration_fp = get_demonstration_fp()
df = pd.read_csv(demonstration_fp)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,"Braund, Mr. Owen Harris",male,22.0,0,7.2500,NaN,S,1937-10-28,15:53:04,2022-08-05 04:43:34,NaN
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,0,71.2833,C85,C,NaN,12:26:00,2022-08-07 01:56:33,NaN
2,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,NaN,S,1931-09-24,16:08:25,2022-08-04 20:27:37,NaN
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,1936-11-30,NaN,2022-08-07 07:05:55,NaN
4,5,"Allen, Mr. William Henry",male,35.0,0,8.0500,NaN,S,1918-11-07,10:59:08,2022-08-02 15:13:34,NaN


MetaSynth will automatically generate the metadata from this DataFrame object so it is important to __ensure the data types for all the variables are correct__. For example, in the dataset above we see that Age is a floating point number whereas it should be an integer (22 instead of 22.0). In addition, there are a few categorical variables (Sex, Parch, Embarked) which are loaded in as string data types.

In general, we support [pandas dtypes](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes). For our example dataset we can specify the `dtypes` and load the dataset as follows:

In [4]:
dtypes = {
    "Survived": "category",  # Categories should be assigned this type.
    "Name": "string",  # Strings should be assigned like this
    "Age": "Int64",  # Integer columns that have NA's in them should be explicitly nullable integers.
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category",
}
df = pd.read_csv(demonstration_fp, dtype=dtypes, parse_dates=["Married since"])  # Parse datetimes with parse_dates
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,"Braund, Mr. Owen Harris",male,22,0,7.2500,<NA>,S,1937-10-28,15:53:04,2022-08-05 04:43:34,NaN
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,0,71.2833,C85,C,NaN,12:26:00,2022-08-07 01:56:33,NaN
2,3,"Heikkinen, Miss. Laina",female,26,0,7.9250,<NA>,S,1931-09-24,16:08:25,2022-08-04 20:27:37,NaN
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,0,53.1000,C123,S,1936-11-30,NaN,2022-08-07 07:05:55,NaN
4,5,"Allen, Mr. William Henry",male,35,0,8.0500,<NA>,S,1918-11-07,10:59:08,2022-08-02 15:13:34,NaN


### Dates and times

While pandas can easily parse datetime columns, it cannot parse proper times and dates. Instead, we use the types from the built-in `datetime` package. Thus we have to manually transform the strings in the columns with date, time, and datetimes to their proper objects. Since the columns in our example dataset follow the standard ISO-format, we can convert them with the `fromisoformat` method. If they are written in a different format, check out the [datetime library documentation](https://docs.python.org/3/library/datetime.html) on how to convert the strings to datetime/time/date objects.

Note the check for NA dates/times, otherwise it will raise an error instead.

In [5]:
df["Birthday"] = [dt.date.fromisoformat(x) if not pd.isna(x) else pd.NA for x in df["Birthday"]]
df["Board time"] = [dt.time.fromisoformat(x) if not pd.isna(x) else pd.NA for x in df["Board time"]]

Now, let's check the data types of our DataFrame:

In [6]:
df.dtypes

PassengerId               int64
Name                     string
Sex                    category
Age                       Int64
Parch                  category
Fare                    float64
Cabin                    string
Embarked               category
Birthday                 object
Board time               object
Married since    datetime64[ns]
all_NA                  float64
dtype: object

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [7]:
df["Birthday"][0]

datetime.date(1937, 10, 28)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [8]:
meta_dataset = MetaDataset.from_dataframe(df)

/home/qubix/Documents/shared_work/synthetic/meta-synth/metasynth/disttree.py:127: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [9]:
print(meta_dataset)

# Rows: 891
# Columns: 12

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameter

Note that the column with all NA's has been converted to an exponential distribution. This has no effect on the outcome, since it will generate NA's exclusively.

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [10]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [11]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,192,"fY LUQfL,sh%FC@2A JhF""ruDN/i?y)1O/GcOicC2 NGp&...",female,14,0,3.984236,<NA>,S,1903-11-28,10:47:25,2022-08-06 08:46:13,NaN
1,429,7ki3De(i` w#/7XTbHN#dt[-CE[usq# MDJfT~v@Q}7:Vb...,female,52,1,102.994357,AvIN~]yp1bI,C,1921-01-14,13:09:26,NaT,NaN
2,10,@&VULF$ke@Y2}En#9u0{8^AdgJ$X?\~5ci{2{6\;5sZq;-...,male,73,2,37.089501,AL|!!M\c,S,1914-08-11,17:00:37,2022-08-13 19:03:09,NaN
3,495,"Gfxo@MG""Yr Tz4)@+KSL",male,<NA>,2,15.836109,TX%,S,None,15:48:29,2022-07-21 04:49:27,NaN
4,479,"s+-\""Ft5\$4Qq-2&q l",male,33,0,5.815324,<NA>,C,1910-08-23,18:06:40,2022-08-13 10:34:45,NaN


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [12]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'int64',
 'prop_missing': 0.0,
 'distribution': {'name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [13]:
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,u83eZ~L lY;<V|,female,11,0,35.788838,<NA>,S,1912-06-10,13:48:18,NaT,NaN
1,2,Gat{fwp lt) !I <M9'g!g!R,female,10,2,8.482919,<NA>,S,1939-03-10,15:03:24,2022-08-02 21:06:07,NaN
2,3,}S$!Mq}UMn@.uQ&cHX'/1|hJ++O@zBub78YnJ0@/]~O}|Q...,male,55,0,26.752672,<NA>,S,1917-05-31,12:06:59,2022-07-18 14:21:02,NaN
3,4,"Q[UO9&OrQ 6o?Jk p3,25kEhmka5Ua?z''}'cH]pc_l|u...",female,27,0,25.390086,<NA>,S,1905-10-02,18:35:02,2022-08-02 16:42:06,NaN
4,5,"stXg,-mF GPUj//",male,20,0,65.646874,<NA>,S,1938-11-17,15:01:59,2022-07-19 13:10:52,NaN


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [14]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,Andrew Figueroa,male,75,0,100.006918,<NA>,S,1940-04-05,13:22:56,2022-07-20 18:51:55,NaN
1,2,Paul Wright,male,52,0,8.296979,<NA>,S,1926-12-21,13:47:03,NaT,NaN
2,3,Michele Nguyen,male,69,0,82.728196,<NA>,S,1929-05-13,12:07:56,2022-08-01 01:09:25,NaN
3,4,Bradley Patel,male,48,0,13.914581,Ck__El Oo{,S,1917-03-19,13:11:42,2022-07-18 15:11:19,NaN
4,5,Molly Floyd,male,59,0,7.152173,<NA>,S,1924-08-16,15:11:41,2022-07-23 00:46:23,NaN


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [15]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,Jessica Taylor,male,28,0,0.809322,<NA>,C,1928-06-21,17:42:39,2022-08-02 14:18:50,NaN
1,2,Stephanie Herrera,male,28,0,1.151924,<NA>,C,1913-04-02,15:52:40,2022-08-03 10:09:50,NaN
2,3,Joshua Butler,female,<NA>,0,0.891668,<NA>,C,None,18:09:44,2022-08-13 22:29:04,NaN
3,4,Mallory Chavez,female,36,2,1.844814,<NA>,S,1934-04-08,18:34:03,2022-08-10 20:26:18,NaN
4,5,Yolanda Moran,female,35,0,1.486541,<NA>,Q,1925-07-07,14:22:19,2022-08-12 22:24:25,NaN


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [16]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[ABCDEF]\d{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name", "prop_missing": 0.1},  # Manually set the proportion of missing values.
    "Fare": {"distribution": "LogNormalDistribution"},  # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)},  # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,Mary Schroeder,male,27,0,0.882189,<NA>,S,1916-04-03,13:18:23,2022-07-22 04:23:08,NaN
1,2,Jennifer Gonzalez,male,<NA>,0,0.358109,<NA>,S,1933-05-11,13:44:39,2022-07-25 02:32:03,NaN
2,3,Marcus Davis,female,27,0,2.185929,<NA>,S,1924-07-30,17:45:23,2022-08-03 00:05:42,NaN
3,4,Robin Walker,male,21,0,2.383195,<NA>,S,1914-10-23,17:13:26,2022-08-09 14:12:14,NaN
4,5,Daniel Hanna,male,<NA>,0,0.660969,<NA>,S,1915-08-15,14:13:36,2022-08-05 01:37:41,NaN
